In [ ]:
import pandas as pd
import numpy as np
import re
from harvesttext import HarvestText
import os

In [ ]:
data=pd.read_csv(r'../../Data/input/Weibo_uncleaned_data.csv')

All data clean

In [ ]:
# Removal of duplicate data
data=data.drop_duplicates(subset=['连接.1'],keep='last')
# Filtering data containing related words
keywords = ['城市空中交通', '低空', '飞行汽车','空中的士','eVTOL','evtol','电动垂直起降机','空中出租车','空中物流','低空出行','空中快递','物流无人机','无人机配送','空中出勤','空中通勤','飞行出租车','空中出租车','外卖无人机','无人驾驶航空器','大疆','丰翼科技','亿航','道通智能','路飞智能']
data = data[data["微博正文"].str.contains('|'.join(keywords))]
# Remove the data containing completely unrelated words
keywords = ['航拍','车','概念股','上证','A股','科创版','深指','沪指']
data=data[~data["微博正文"].str.contains('|'.join(keywords))]

In [ ]:
data["cleaned_text"] = data["微博正文"].str.lstrip() #只删除句首所有空格
# Extract posting year
data['year'] = data['Unnamed: 10'].str.slice(0, 4)
# Remove useless characters from text
data["cleaned_text"] = data["cleaned_text"].str.replace(" L.*?微博视频", "", regex=True).replace(" L.*?秒拍视频", "", regex=True)\
.replace("（.*?）", "", regex=True).replace("\(.*?\)", "", regex=True).replace(" 2.*? ", "", regex=True).replace(" O.*? ", "", regex=True)\
.replace("\d{11}|\d{8}|\d{4}-\d{7}|\d{3}-\d{8}", "", regex=True).replace("。O.*? ", "", regex=True).replace("？O.*? ", "", regex=True).replace("！O.*? ", "", regex=True)\
.replace("L.*?微博视频", "", regex=True).replace("L.*?秒拍视频", "", regex=True).replace("L.*?快手视频", "", regex=True)
ht0 = HarvestText()
data["cleaned_text"] = data["cleaned_text"].map(lambda r:ht0.clean_text(r,weibo_at=True,weibo_topic=True,stop_terms=('收起d','O网页链接','秒拍视频','头条播报','微博视频','我发表了头条文章','分享','点击','链接')))
data=data[-(data["cleaned_text"]=='')]
data["cleaned_text"] = data["cleaned_text"].str.replace("【", "", regex=True).replace("】", "", regex=True)
# deleting null values
data=data[-(data["cleaned_text"]=='')]

Keyword "大疆无人机" data clean

In [ ]:
data_dajiang=data[data['关键词']=="大疆无人机"]
keywords = ['运载无人机','FlyCart','货箱','空吊','运输','物流','Flycart','flycart','运载']
data_dajiang = data_dajiang[data_dajiang["text_cleaned"].str.contains('|'.join(keywords))]
keywords = ['暴力运输']
data_dajiang=data_dajiang[-(data_dajiang["text_cleaned"].str.contains('|'.join(keywords)))]

In [ ]:
data=data[-(data['关键词']=='大疆无人机')]
data=pd.concat([data,data_dajiang])

User type merge

In [ ]:
data=data[-(data['verified_type']==5)]
# Integrate all general public users
data['verified_type'].replace([-1,0,400,10,200, 220], '--', inplace=True)
# Integrate all business users
data['verified_type'].replace([2,3,4,6,7,8], 'Business', inplace=True)
# Integrate all government users
data['verified_type'].replace(1, 'Government', inplace=True)

Nation transfer

In [ ]:
data_loca=data.copy()
data_loca['省份']=data_loca['location'].str.extract(r'(\w+)')
data_loca['ip_location']=data_loca['ip_location'].fillna(data_loca['省份'])
data_loca['ip_location']=data_loca['ip_location'].replace({'中国香港':'香港','中国台湾':'台湾','中国澳门':'澳门'})
data_loca['nation'] = np.where(data_loca['ip_location'].isin(['北京','广东','上海','陕西','山东','四川','浙江','江苏','河北','河南','天津','湖北','湖南','辽宁','重庆','福建','安徽','吉林','山西','黑龙江','江西','甘肃','内蒙古','云南','广西','贵州','海南','宁夏','香港','新疆','青海','台湾','西藏','澳门','中国']), 'China', 'Unknown')
data_loca

Retain useful data for next statistical analysis

In [ ]:
data_retained = data_loca[['关键词', '连接.1', '博主名','verified_type','nation','year','cleaned_text']]
useful_data = pd.DataFrame(data_retained)
# Regularization of column names
new_column_names = {'关键词': 'keyword', '连接.1': 'link', '博主名': 'user_id', 'verified_type': 'user_type', 'nation': 'nation', 'year': 'year', 'cleaned_text': 'text_cleaned'}
useful_data = useful_data.rename(columns=new_column_names)

In [ ]:
useful_data.to_csv('../../Data/interim/weibo_text.csv', encoding="utf_8_sig",index=False)